In [6]:
import os
import glob
import cv2
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import zoom

from PIL import Image

In [7]:
def maxmin_norm(data):
    MAX = np.amax(data)
    MIN = np.amin(data)
    data = (data - MIN)/(MAX-MIN)
    return data

In [33]:
import numpy as np

z = 10
n_slice = 5
index = np.zeros((z, n_slice))
print(index)

for idx_z in range(z):
    for idx_c in range(n_slice):
        index[idx_z, idx_c] = idx_z-(n_slice-idx_c+1)+n_slice//2+2
index[index<0]=0
index[index>z-1]=z-1
print(index)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 0. 0. 1. 2.]
 [0. 0. 1. 2. 3.]
 [0. 1. 2. 3. 4.]
 [1. 2. 3. 4. 5.]
 [2. 3. 4. 5. 6.]
 [3. 4. 5. 6. 7.]
 [4. 5. 6. 7. 8.]
 [5. 6. 7. 8. 9.]
 [6. 7. 8. 9. 9.]
 [7. 8. 9. 9. 9.]]


In [ ]:
def create_index(dataA, n_slice):
    h, w, z = dataA.shape
    index = np.zeros((z,n_slice))
    
    for idx_z in range(z):
        for idx_c in range(n_slice):
            index[idx_z, idx_c] = idx_z-(n_slice-idx_c+1)+n_slice//2+2
    index[index<0]=0
    index[index>z-1]=z-1
    return index

In [ ]:
def slice5(dataA, dataB, name_dataset, n_slice=1, name_tag="", resize_f=1):
    # shape supposed to be 512*512*284 by default
    assert dataA.shape == dataB.shape, ("DataA should share the same shape with DataB.")
    path2save = "./pytorch-CycleGAN-and-pix2pix/datasets/"+name_dataset+"/train/"
    h, w, z = dataA.shape
    h = h*resize_f
    w = w*resize_f
    img = np.zeros((n_slice, h, w*2))
    index = create_index(dataA, n_slice)
        
    for idx_z in range(z):
        for idx_c in range(n_slice):
            img[idx_c, :, :w] = zoom(dataA[:, :, index(idx_z, idx_c)], zoom=resize_f)
            img[idx_c, :, w:] = zoom(dataB[:, :, index(idx_z, idx_c)], zoom=resize_f)
        name2save = path2save+name_tag+"_"+str(idx_z)+".npy"
        np.save(name2save, img)
    print(str(c)+" images have been saved.")

In [8]:
def SpotTheDifference_Generator(dataA, dataB, name_dataset, n_slice=1, name_tag="", resize_f=1):
    # shape supposed to be 512*512*284 by default
    assert dataA.shape == dataB.shape, ("DataA should share the same shape with DataB.")
    path2save = "./pytorch-CycleGAN-and-pix2pix/datasets/"+name_dataset+"/train/"
    h, w, c = dataA.shape
    h = h*resize_f
    w = w*resize_f
    img = np.zeros((n_slice, h, w*2))
        
    if n_slice == 1:
        for idx in range(c):
            img[:, :, :w] = zoom(dataA[:, :, idx], zoom=resize_f)
            img[:, :, w:] = zoom(dataB[:, :, idx], zoom=resize_f)
            name2save = path2save+name_tag+"_"+str(idx)+".npy"
            np.save(name2save, img)
        print(str(c)+" images have been saved.")
    else:
        for idx in range(c):
            if idx == 0:
                img[0, :, :w] = zoom(dataA[:, :, 0], zoom=resize_f)
                img[1, :, :w] = zoom(dataA[:, :, 0], zoom=resize_f)
                img[2, :, :w] = zoom(dataA[:, :, 1], zoom=resize_f)
                
                img[0, :, w:] = zoom(dataB[:, :, 0], zoom=resize_f)
                img[1, :, w:] = zoom(dataB[:, :, 0], zoom=resize_f)
                img[2, :, w:] = zoom(dataB[:, :, 1], zoom=resize_f)
            else:
                if idx == c-1:
                    img[0, :, :w] = zoom(dataA[:, :, c-2], zoom=resize_f)
                    img[1, :, :w] = zoom(dataA[:, :, c-1], zoom=resize_f)
                    img[2, :, :w] = zoom(dataA[:, :, c-1], zoom=resize_f)
                    
                    img[0, :, w:] = zoom(dataB[:, :, c-2], zoom=resize_f)
                    img[1, :, w:] = zoom(dataB[:, :, c-1], zoom=resize_f)
                    img[2, :, w:] = zoom(dataB[:, :, c-1], zoom=resize_f)
                else:
                    img[0, :, :w] = zoom(dataA[:, :, idx-1], zoom=resize_f)
                    img[1, :, :w] = zoom(dataA[:, :, idx], zoom=resize_f)
                    img[2, :, :w] = zoom(dataA[:, :, idx+1], zoom=resize_f)
                    
                    img[0, :, w:] = zoom(dataB[:, :, idx-1], zoom=resize_f)
                    img[1, :, w:] = zoom(dataB[:, :, idx], zoom=resize_f)
                    img[2, :, w:] = zoom(dataB[:, :, idx+1], zoom=resize_f)
            name2save = path2save+name_tag+"_"+str(idx)+".npy"
            np.save(name2save, img)
        print(str(c)+" images have been saved.")

In [16]:
def SingleImage_Generator(dataA, name_dataset, n_slice=1, name_tag="", resize_f=1):
    # shape supposed to be 512*512*284 by default
    path2save = "./pytorch-CycleGAN-and-pix2pix/datasets/"+name_dataset+"/test/"
    h, w, c = dataA.shape
    h = h*resize_f
    w = w*resize_f
    img = np.zeros((n_slice, h, w))
        
    if n_slice == 1:
        for idx in range(c):
            img[:, :, :] = zoom(dataA[:, :, idx], zoom=resize_f)
#             img = np.asarray(img, dtype=np.float())
            name2save = path2save+name_tag+"_"+str(idx)+".npy"
            np.save(name2save, img)
        print(str(c)+" images have been saved.")
    else:
        for idx in range(c):
            if idx == 0:
                img[0, :, :] = zoom(dataA[:, :, 0], zoom=resize_f)
                img[1, :, :] = zoom(dataA[:, :, 0], zoom=resize_f)
                img[2, :, :] = zoom(dataA[:, :, 1], zoom=resize_f)
            else:
                if idx == c-1:
                    img[0, :, :] = zoom(dataA[:, :, c-2], zoom=resize_f)
                    img[1, :, :] = zoom(dataA[:, :, c-1], zoom=resize_f)
                    img[2, :, :] = zoom(dataA[:, :, c-1], zoom=resize_f)
                else:
                    img[0, :, :] = zoom(dataA[:, :, idx-1], zoom=resize_f)
                    img[1, :, :] = zoom(dataA[:, :, idx], zoom=resize_f)
                    img[2, :, :] = zoom(dataA[:, :, idx+1], zoom=resize_f)
            name2save = path2save+name_tag+"_"+str(idx)+".npy"
#             img = np.asarray(img, dtype=np.float())
            np.save(name2save, img)
        print(str(c)+" images have been saved.")

In [17]:
name_dataset = "pet_test"
n_slice = 3

import os

for folder_name in ["train", "test", "trainA", "trainB", "testA", "testB"]:
    path = "./pytorch-CycleGAN-and-pix2pix/datasets/"+name_dataset+"/"+folder_name+"/"
    if not os.path.exists(path):
        os.makedirs(path)

## test dataset of UNET

In [18]:
list_ori = glob.glob("./data/"+name_dataset+"/test/*.nii")
list_ori.sort()
print("Test:")
for path_ori in list_ori:
    filename_ori = os.path.basename(path_ori)[:]
    filename_ori = filename_ori[:filename_ori.find(".")]
    print(filename_ori)
    data_ori = maxmin_norm(nib.load(path_ori).get_fdata())
    SingleImage_Generator(dataA=data_ori, name_dataset=name_dataset, n_slice=n_slice, name_tag=filename_ori,
                          resize_f = 2)
    print("------------------------------------------------------------------------")

Test:
subj01_pet
284 images have been saved.
------------------------------------------------------------------------


## training dataset  of UNET

In [31]:
list_ori = glob.glob("./data/"+name_dataset+"/pure/*.nii")
list_ori.sort()
for path_ori in list_ori:
    print("TrainA:")
    filename_ori = os.path.basename(path_ori)[:]
    filename_ori = filename_ori[:filename_ori.find(".")]
    print(filename_ori)
    data_ori = maxmin_norm(nib.load(path_ori).get_fdata())
    
    list_sim = glob.glob("./data/"+name_dataset+"/blur/*"+filename_ori+"*.nii")
    list_sim.sort()
    
    for path_sim in list_sim:
        print("Pairs")
        filename_sim = os.path.basename(path_sim)[:]
        filename_sim = filename_sim[:filename_sim.find(".")]
        print("A:", filename_ori)
        print("B:", filename_sim)
                
        data_sim = maxmin_norm(nib.load(path_sim).get_fdata())
        SpotTheDifference_Generator(dataA=data_ori, dataB=data_sim,
                                    name_dataset=name_dataset, n_slice=n_slice, 
                                    name_tag=filename_sim, resize_f=2)
        
    print("------------------------------------------------------------------------")
        
        
        
#         data_ori = nib.load(path_ori).get_fdata()
#         norm_ori = maxmin_norm(data_ori)*255
#         sliced_save(data=norm_ori,
#                     name_tag=os.path.basename(path_ori)[:-4],
#                     path2save="./pytorch-CycleGAN-and-pix2pix/datasets/"+name_dataset+"/"+folder_name+"/",
#                     n_slice=n_slice)

TrainA:
subj01_inv
Pairs
A: subj01_inv
B: fwhm_8_subj01_inv
284 images have been saved.
Pairs
A: subj01_inv
B: fwhm_8_subj01_inv_gs_1e-02
284 images have been saved.
Pairs
A: subj01_inv
B: fwhm_8_subj01_inv_gs_3e-02


KeyboardInterrupt: 

## training dataset of cycleGAN

In [6]:
def cycleGAN_generator(dataA, dataB, name_dataset, n_slice=1, name_tag=""):
    # shape supposed to be 512*512*284 by default
    assert dataA.shape == dataB.shape, ("DataA should share the same shape with DataB.")
    path2saveA = "./pytorch-CycleGAN-and-pix2pix/datasets/"+name_dataset+"/trainA/"
    path2saveB = "./pytorch-CycleGAN-and-pix2pix/datasets/"+name_dataset+"/trainB/"
    h, w, z = dataA.shape
    imgA = np.zeros((n_slice, h, w))
    imgB = np.zeros((n_slice, h, w))
    
    if n_slice == 1:
        for idx in range(z):
            imgA[0, :, :] = dataA[:, :, idx]
            imgB[0, :, :] = dataB[:, :, idx]
            name2saveA = path2saveA+name_tag+"_"+str(idx)+".npy"
            name2saveB = path2saveB+name_tag+"_"+str(idx)+".npy"
            np.save(name2saveA, imgA)
            np.save(name2saveB, imgB)
        print(str(z)+" images have been saved.")
    else:
        for idx in range(z):
            if idx == 0:
                imgA[0, :, :] = dataA[:, :, 0]
                imgA[1, :, :] = dataA[:, :, 0]
                imgA[2, :, :] = dataA[:, :, 1]
                
                imgB[0, :, :] = dataB[:, :, 0]
                imgB[1, :, :] = dataB[:, :, 0]
                imgB[2, :, :] = dataB[:, :, 1]
            else:
                if idx == z-1:
                    imgA[0, :, :] = dataA[:, :, z-2]
                    imgA[1, :, :] = dataA[:, :, z-1]
                    imgA[2, :, :] = dataA[:, :, z-1]
                    
                    imgB[0, :, :] = dataB[:, :, z-2]
                    imgB[1, :, :] = dataB[:, :, z-1]
                    imgB[2, :, :] = dataB[:, :, z-1]
                else:
                    imgA[0, :, :] = dataA[:, :, idx-1]
                    imgA[1, :, :] = dataA[:, :, idx]
                    imgA[2, :, :] = dataA[:, :, idx+1]
                    
                    imgB[0, :, :] = dataB[:, :, idx-1]
                    imgB[1, :, :] = dataB[:, :, idx]
                    imgB[2, :, :] = dataB[:, :, idx+1]
            name2saveA = path2saveA+name_tag+"_"+str(idx)+".npy"
            name2saveB = path2saveB+name_tag+"_"+str(idx)+".npy"
            np.save(name2saveA, imgA)
            np.save(name2saveB, imgB)
        print(str(z)+" images have been saved.")

In [7]:
list_ori = glob.glob("./data/"+name_dataset+"/pure/*.nii")
list_ori.sort()
for path_ori in list_ori:
    print("TrainA:")
    filename_ori = os.path.basename(path_ori)[:]
    filename_ori = filename_ori[:filename_ori.find(".")]
    print(filename_ori)
    data_ori = maxmin_norm(nib.load(path_ori).get_fdata())
    
    list_sim = glob.glob("./data/"+name_dataset+"/blur/*"+filename_ori+"*.nii")
    list_sim.sort()
    
    for path_sim in list_sim:
        print("Pairs")
        filename_sim = os.path.basename(path_sim)[:]
        filename_sim = filename_sim[:filename_sim.find(".")]
        print("A:", filename_ori)
        print("B:", filename_sim)
                
        data_sim = maxmin_norm(nib.load(path_sim).get_fdata())
        cycleGAN_generator(dataA=data_ori, dataB=data_sim,
                           name_dataset=name_dataset, n_slice=n_slice, name_tag=filename_sim)
        
    print("------------------------------------------------------------------------")
        
        
        
#         data_ori = nib.load(path_ori).get_fdata()
#         norm_ori = maxmin_norm(data_ori)*255
#         sliced_save(data=norm_ori,
#                     name_tag=os.path.basename(path_ori)[:-4],
#                     path2save="./pytorch-CycleGAN-and-pix2pix/datasets/"+name_dataset+"/"+folder_name+"/",
#                     n_slice=n_slice)

TrainA:
subj01_inv
Pairs
A: subj01_inv
B: fwhm_8_subj01_inv
284 images have been saved.
Pairs
A: subj01_inv
B: fwhm_8_subj01_inv_radon
284 images have been saved.
------------------------------------------------------------------------


In [25]:
A = np.ones((3,3))
print(A)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [26]:
B = np.ones((3,3))*8
print(B)

[[8. 8. 8.]
 [8. 8. 8.]
 [8. 8. 8.]]


In [27]:
C = np.zeros((6,3))
C[:3,:]=A
C[3:,:]=B
print(C)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [8. 8. 8.]
 [8. 8. 8.]
 [8. 8. 8.]]


In [20]:
a = np.ones((2,3))
print(a)
b = zoom(a, 2)
print(b)

[[1. 1. 1.]
 [1. 1. 1.]]
[[1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]]
